In [ ]:
from rich.console import Console
import mlflow
from prompt_toolkit import prompt
from prompt_toolkit.history import InMemoryHistory
from prompt_toolkit.styles import Style
from dotenv import load_dotenv
import openai

load_dotenv()

MODEL = "gpt-oss-20b-local"
PROMPTS = "prompts:/qa-agent-user-prompt@latest"
TEMPERATURE = 0.1


# Step 1: Register your initial prompt
# 建立 user prompt，如果已存在則不重複建立
def create_or_load_prompt():
    user_prompt = mlflow.genai.load_prompt(PROMPTS)
    if not user_prompt:
        mlflow.genai.register_prompt(
            name="qa-agent-user-prompt",
            template="""You are a helpful assistant. Answer questions concisely in Traditional Chinese.
Question: {{question}}
Answer:""",
        )
        print("Prompt not found. Registered a new prompt.")
    return mlflow.genai.load_prompt(PROMPTS)


# Step 2: Create a prediction function
def predict_fn(question: str) -> str:
    # Load prompt from registry
    user_prompt = create_or_load_prompt()
    formatted_prompt = user_prompt.format(question=question)

    # 檢查 formatted_prompt 是列表還是字串
    if isinstance(formatted_prompt, list):
        messages = formatted_prompt
    else:
        messages = [{"role": "user", "content": formatted_prompt}]

    response = openai.OpenAI().chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=TEMPERATURE,
    )
    return response.choices[0].message.content


# # Step 3: Define scorers
# print(create_or_load_prompt().template)

input_style = Style.from_dict(
    {
        "prompt": "cyan bold",
    }
)

console = Console()
history = InMemoryHistory()
console = Console()


def get_user_input() -> str:
    """取得使用者輸入"""
    return prompt(
        [("class:prompt", "👤 你: ")],
        style=input_style,
        history=history,
        mouse_support=True,
    ).strip()


def display_response(text: str):
    """顯示助手回應"""
    console.print(f"[bold green]🤖 助手:[/bold green] {text}\n")


while True:
    try:
        question = get_user_input()
        if question.lower() == "exit":
            console.print("退出程式。")
            break
        result = predict_fn(question)
        display_response(result)
    except KeyboardInterrupt:
        console.print("\n\n👋 再見！")
        break
    except Exception as e:
        console.print(f"[red]❌ 錯誤: {e}[/red]\n")
